In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

FILENAME = "C:/Users/Yi Yang/Documents/Dsci521/Video_Games_Sales_as_at_22_Dec_2016.csv" 
df= pd.read_csv(FILENAME, na_values='tbd')



In [48]:
df.columns


Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')

In [49]:
df['Name'].fillna('Unknown', inplace=True)
df['Platform'].fillna('Unknown', inplace=True)
df['Year_of_Release'].fillna(df['Year_of_Release'].median(), inplace=True)
df['Genre'].fillna('Unknown', inplace=True)
df['Publisher'].fillna('Unknown', inplace=True)
df['Developer'].fillna('Unknown', inplace=True)
df['Rating'].fillna('Unknown', inplace=True)

# Year_of_Release to int
df['Year_of_Release'] = df['Year_of_Release'].astype(int)

# handle outliers
df['NA_Sales'] = df['NA_Sales'].clip(lower=0)
df['EU_Sales'] = df['EU_Sales'].clip(lower=0)
df['JP_Sales'] = df['JP_Sales'].clip(lower=0)
df['Other_Sales'] = df['Other_Sales'].clip(lower=0)
df['Global_Sales'] = df['Global_Sales'].clip(lower=0)

# Critic_Score，Critic_Count，User_Score User_Count lossing value
df['Critic_Score'].fillna(df['Critic_Score'].mean(), inplace=True)
df['Critic_Count'].fillna(df['Critic_Count'].mean(), inplace=True)
df['User_Score'].fillna(df['User_Score'].mean(), inplace=True)
df['User_Count'].fillna(df['User_Count'].mean(), inplace=True)

# Critic_Count User_Count to ine
df['Critic_Count'] = df['Critic_Count'].astype(int)
df['User_Count'] = df['User_Count'].astype(int)

In [50]:
df.info

<bound method DataFrame.info of                                 Name Platform  Year_of_Release         Genre  \
0                         Wii Sports      Wii             2006        Sports   
1                  Super Mario Bros.      NES             1985      Platform   
2                     Mario Kart Wii      Wii             2008        Racing   
3                  Wii Sports Resort      Wii             2009        Sports   
4           Pokemon Red/Pokemon Blue       GB             1996  Role-Playing   
...                              ...      ...              ...           ...   
16714  Samurai Warriors: Sanada Maru      PS3             2016        Action   
16715               LMA Manager 2007     X360             2006        Sports   
16716        Haitaka no Psychedelica      PSV             2016     Adventure   
16717               Spirits & Spells      GBA             2003      Platform   
16718            Winning Post 8 2016      PSV             2016    Simulation   

       

In [51]:
negative_sales = df[df['Global_Sales'] < 0]
if not negative_sales.empty:
    mean_sales = df[df['Global_Sales'] >= 0]['Global_Sales'].mean()
    df['Global_Sales'] = df['Global_Sales'].apply(lambda x: mean_sales if x < 0 else x)


## Sales ratio by region
df['Sales_Per_User'] = df['Global_Sales'] / df['User_Count']
df['NA_Sales_Ratio'] = df['NA_Sales'] / df['Global_Sales']
df['EU_Sales_Ratio'] = df['EU_Sales'] / df['Global_Sales']
df['JP_Sales_Ratio'] = df['JP_Sales'] / df['Global_Sales']
df['Other_Sales_Ratio'] = df['Other_Sales'] / df['Global_Sales']
# Multiply user ratings by 10 to have a similar scale to critic ratings

df['Score_Difference'] = df['Critic_Score'] - df['User_Score'] * 10  
## Calculates the weighted average of user ratings and critic ratings to get a composite rating.
critic_weight = 0.6  
user_weight = 0.4
df['Total_Score'] = (df['Critic_Score'] * critic_weight) + (df['User_Score'] * 10 * user_weight)

## Game age
current_year = 2023
df['Game_Age'] = current_year - df['Year_of_Release']


df.columns


Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating',
       'Sales_Per_User', 'NA_Sales_Ratio', 'EU_Sales_Ratio', 'JP_Sales_Ratio',
       'Other_Sales_Ratio', 'Score_Difference', 'Total_Score', 'Game_Age'],
      dtype='object')

In [52]:
df['Platform_Publisher'] = df['Platform'] + '_' + df['Publisher']
df['Score_Percentage_Difference'] = (df['Critic_Score'] - df['User_Score'] * 10) / df['Critic_Score'] * 100
df['Normalized_Critic_Score'] = (df['Critic_Score'] - df['Critic_Score'].min()) / (df['Critic_Score'].max() - df['Critic_Score'].min())
df['Normalized_User_Score'] = (df['User_Score'] - df['User_Score'].min()) / (df['User_Score'].max() - df['User_Score'].min())
df['Sales_Critic_Score_Ratio'] = df['Global_Sales'] / df['Critic_Score']
df['Sales_User_Score_Ratio'] = df['Global_Sales'] / (df['User_Score'] * 10)


In [53]:
## Normalize numeric features:
num_features = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Sales_Per_User']
scaler = StandardScaler()
df[num_features] = scaler.fit_transform(df[num_features])
## Group by game name and apply an aggregate function to combine duplicate game entries into a single entry
df.sort_values('Global_Sales', ascending=False, inplace=True)






In [54]:
# 检查各个地区销售额是否为 0
zero_sales = df[(df["NA_Sales"] == 0) & (df["EU_Sales"] == 0) & (df["JP_Sales"] == 0) & (df["Other_Sales"] == 0)]
print("销售额为 0 的记录：\n", zero_sales)

# 如果存在销售额为 0 的记录，可以选择删除
if not zero_sales.empty:
    df = df[~((df["NA_Sales"] == 0) & (df["EU_Sales"] == 0) & (df["JP_Sales"] == 0) & (df["Other_Sales"] == 0))]

cleaned_csv_filename = "C:/Users/Yi Yang/Documents/Dsci521/cleaned_vgames_data.csv" 
df.to_csv(cleaned_csv_filename, index=False)